In [1]:
import pandas as pd
import numpy as np
import time
import importlib.machinery
es = importlib.machinery.SourceFileLoader('extrasense','/home/sac086/extrasensory/extrasense/extrasense.py').load_module()
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler

# Testing a Personal Model
This is just so that I can get a sense of whether the data is working out.

In [2]:
def test_personal_model(user_id):
    kf = KFold(n_splits=3)

    user_df = es.get_data_from_user_id(user_id, data_type="activity", labeled_only=True)
    timestamps_series = user_df.pop('timestamp')
    scores = []
    for training_ind, test_ind in kf.split(user_df):
        train_df = user_df.iloc[training_ind]
        test_df = user_df.iloc[test_ind]

        train_labels = train_df.pop("label")
        test_labels = test_df.pop("label")
        

        clf = RandomForestClassifier()

        # standard scale training
        
        scaled_test_df = scaler.transform(test_df)

        clf.fit(scaled_train_df, train_labels)

        # standard scale testing
        predictions = clf.predict(scaled_test_df)
        score = accuracy_score(test_labels, predictions)
        scores.append(score)
    return np.mean(scores)

In [3]:
#user_scores = {}
#for user_id in es.user_ids:
#    mean_score = test_personal_model(user_id)
#    user_scores[user_id] = mean_score

# Testing an impersonal model
Leave-One Participant out design

Sampling personal test data with ShuffleSplit method which will randomly select personal data
making no gaurentee that the distribution of test data is similar to the distribution of training data

In [4]:
user_df = es.get_data_from_user_id(es.user_ids[0], data_type="activity", labeled_only=True)

/home/sac086/extrasensory/extrasense/extrasense.py:159: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  user_df = user_df[null_mask]


In [5]:
impersonal_df = es.get_impersonal_data(es.user_ids[0], data_type="activity", labeled_only=True)

/home/sac086/extrasensory/extrasense/extrasense.py:181: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  impersonal_df = impersonal_df[null_mask]


In [ ]:
impersonal_df = pd.concat(impersonal_df, ignore_index=True)

In [ ]:
features_df = impersonal_df[impersonal_df.columns.intersection(es.acc_col_name)]

In [ ]:
labels_df = es.clean_labels(impersonal_df[impersonal_df.columns.intersection(es.acc_labels_name)])

In [ ]:
impersonal_df = pd.concat((features_df,labels_df,impersonal_df['timestamp']), axis=1)

In [ ]:
impersonal_df.head()

In [ ]:
type(impersonal_df)

In [ ]:
list(impersonal_df.columns)

In [25]:
def test_impersonal_model(user_id):
    personal_df = es.get_data_from_user_id(user_id, data_type="activity", labeled_only=True)
    impersonal_df = es.get_impersonal_data(user_id, data_type="activity", labeled_only=True)
    timestamps_personal = personal_df.pop('timestamp')
    timestamps_impersonal = impersonal_df.pop('timestamp')
    scores = []
    
    rs = ShuffleSplit(n_splits=3, test_size=100)
    train_df = impersonal_df
    train_labels = train_df.pop("label")
    
    # standard scale training
    scaler = StandardScaler().fit(train_df)
    scaled_train_df = scaler.transform(train_df)
    
    impersonal_clf = RandomForestClassifier()
    impersonal_clf.fit(scaled_train_df, train_labels)

    for __, test_ind in rs.split(personal_df):
        test_df = personal_df.iloc[test_ind]
        
        #print(train_df['label'])
        test_labels = test_df.pop("label")

        scaled_test_df = scaler.transform(test_df)

        # standard scale testing
        predictions = impersonal_clf.predict(scaled_test_df)
        score = accuracy_score(test_labels, predictions)
        scores.append(score)
    return np.mean(scores)

In [26]:
user_scores = {}
for user_id in es.user_ids:
    print("Getting score for %s" % user_id)
    start = time.time()
    mean_score = test_impersonal_model(user_id)
    finish = time.time()
    duration_in_minutes = (finish - start) / 60.
    print("\t M=%s, took %s seconds" % (mean_score, duration_in_minutes))
    user_scores[user_id] = mean_score

Getting score for 098A72A5-E3E5-4F54-A152-BBDA0DF7B694


/home/sac086/extrasensory/extrasense/extrasense.py:159: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  user_df = user_df[null_mask]
/home/sac086/extrasensory/extrasense/extrasense.py:181: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  impersonal_df = impersonal_df[null_mask]


	 M=0.57, took 1.6683868169784546 seconds
Getting score for 0A986513-7828-4D53-AA1F-E02D6DF9561B


/home/sac086/extrasensory/extrasense/extrasense.py:181: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  impersonal_df = impersonal_df[null_mask]


	 M=0.66, took 1.6724058508872985 seconds
Getting score for 0BFC35E2-4817-4865-BFA7-764742302A2D


...

	 M=0.623333333333, took 1.6826470255851746 seconds
Getting score for F50235E0-DD67-4F2A-B00B-1F31ADA998B9


/home/sac086/extrasensory/extrasense/extrasense.py:159: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  user_df = user_df[null_mask]
/home/sac086/extrasensory/extrasense/extrasense.py:181: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  impersonal_df = impersonal_df[null_mask]


	 M=0.513333333333, took 1.6815760334332783 seconds
Getting score for FDAA70A1-42A3-4E3F-9AE3-3FDA412E03BF


/home/sac086/extrasensory/extrasense/extrasense.py:181: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  impersonal_df = impersonal_df[null_mask]


	 M=0.636666666667, took 1.6827624360720317 seconds


In [30]:
np.mean([v for v in user_scores.values()])

0.57072222222222235

In [8]:
import time

In [9]:
time.time()

1512154353.2834132

In [12]:
label_df = impersonal_df.pop('label')